In [19]:
## Load any changes to local modules
%load_ext autoreload
%autoreload 2

import os
import sys

pwd = %pwd
project_dir = '{0}/../../../'.format(pwd)
module_path = os.path.abspath(os.path.join(project_dir))
if module_path not in sys.path:
    sys.path.append(module_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import re
import numpy as np
%matplotlib inline
from IPython.display import display

from nbcpact import AnalyzeQuantCompare,Peptide,PeptideGroup,UcbreUtils


In [22]:
df = pd.read_excel("../../../../../../projects/nb-cpact/KEA_EN80/Analysis/KEA_isoTOP_DN_High_Med_All.xlsx")

In [33]:
def __process_modifications(row):
    
    


data = {}

## Clean up dataframe
df = df[~df['Abundance Ratio (log2): (IsoTOP Light) / (IsoTOP Heavy)'].isnull()]

## Process Mods
data['mod_locs'] = df.apply(axis=1, self.__process_modifications)

display(df.head().T)


,55,57,62,64,141
Checked,False,False,False,False,False
Confidence,High,High,High,High,High
Sequence,CGEEIAVQFVDMVK,NNDLCYWVPELVR,TAFLAEDFNPEEINLDCTNPR,SELECVTNITLANVIR,IDATQVEVNPFGETPEGQVVCFDAK
Modifications,1xisoTOP TEV Light [C1],1xisoTOP TEV Light [C5],1xisoTOP TEV Light [C17],1xisoTOP TEV Light [C5],1xisoTOP TEV heavy [C21]
Contaminant,True,True,True,True,True
Qvality PEP,3.29534e-06,0.000654009,2.33843e-07,0.000289251,1.13553e-07
Qvality q-value,0,0,0,0,0
# Protein Groups,1,1,1,1,1
# Proteins,1,1,1,1,1
# PSMs,2,1,3,1,1


In [30]:
df.dropna().T

,55,57,62,64,141,164,170,215,216,239,...,25001,25017,25021,25044,25059,25061,25120,25123,25143,25159
Checked,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Confidence,High,High,High,High,High,High,High,High,High,High,...,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium
Sequence,CGEEIAVQFVDMVK,NNDLCYWVPELVR,TAFLAEDFNPEEINLDCTNPR,SELECVTNITLANVIR,IDATQVEVNPFGETPEGQVVCFDAK,TNMLLQLDGSTPICEDIGR,SLDLFNCEVTNLNDYR,LALFNPDVCWDR,LALFNPDVCWDR,ISEQIVIGTPGTVLDWCSK,...,VSRPLELCYAPTKDPK,ECPSSHPEPTR,MGAVPCR,NHCGIASAASYPTV,MFFIQACR,ENVPSHLPQVTVTQSCVK,KITGLCQEEHR,RVCEEIAIIPSKK,KPVTDCVISVPSFFTDAERR,GCRLSPR
Modifications,1xisoTOP TEV Light [C1],1xisoTOP TEV Light [C5],1xisoTOP TEV Light [C17],1xisoTOP TEV Light [C5],1xisoTOP TEV heavy [C21],1xisoTOP TEV Light [C14],1xisoTOP TEV heavy [C7],1xisoTOP TEV Light [C9],1xisoTOP TEV heavy [C9],1xisoTOP TEV Light [C17],...,1xisoTOP TEV Light [C8],1xisoTOP TEV Light [C2],1xisoTOP TEV Light [C6],1xisoTOP TEV Light [C3],1xisoTOP TEV heavy [C7],1xisoTOP TEV Light [C16],1xisoTOP TEV Light [C6],1xisoTOP TEV Light [C3],1xisoTOP TEV Light [C6],1xisoTOP TEV heavy [C2]
Contaminant,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
Qvality PEP,3.29534e-06,0.000654009,2.33843e-07,0.000289251,1.13553e-07,1.17895e-07,2.75732e-06,1.35872e-05,0.000978654,0.000109046,...,0.544534,0.547329,0.547329,0.550126,0.552926,0.552926,0.561335,0.561335,0.564141,0.566949
Qvality q-value,0,0,0,0,0,0,0,0,0,0,...,0.0448957,0.04521,0.04521,0.0457711,0.0460598,0.0460598,0.047966,0.047966,0.0484875,0.0488418
# Protein Groups,1,1,1,1,1,1,1,1,1,2,...,1,1,1,1,1,1,1,1,1,1
# Proteins,1,1,1,1,1,1,2,1,1,3,...,1,1,1,1,1,1,1,1,1,1
# PSMs,2,1,3,1,1,2,2,6,2,3,...,1,1,1,1,1,1,1,1,1,2
